#  Implementação de um Agente em Flappy Bird

### Imports

In [4]:
import gym
from gym import spaces
import gym_ple
import numpy as np

## Flappy Bird Sem Visão Computacional

### Ambiente

Modelagem do ambiente do Flappy Bird usando o GameState como observação.

#### Observação:
 * Posição Y do pássaro.
 * Velocidade Y do pássaro.
 * Distância do pássaro até o próximo cano.
 * Posição Y da parte de cima do próximo cano.
 * Posição Y da parte de baixo do próximo cano.
 * Distância do pássaro até o cano depois do próximo cano.
 * Posição Y da parte de cima do cano depois do próximo cano.
 * Posição Y da parte de baixo do cano depois do próximo cano.

In [5]:
class FlappyBirdEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array']}

  def __init__(self):
    self.env = gym.make("FlappyBird-v0")
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions:
    self.action_space = self.env.action_space
    # Example for using image as input:
    self.observation_space = spaces.Box(low=np.array([0, -10.0, 0, 0, 0, 0, 0, 0]),
                                        high=np.array([512, 10.0, 588.0, 512, 512, 588.0, 512, 512]),
                                        dtype=np.float32)

  def step(self, action):
    observation, reward, done, info = self.env.step(action)
    observation = np.array(list(self.env.game_state.getGameState().values()))
    if done:
        reward = -1
    reward += 0.1
#     reward += (75 - abs(observation[0] - (observation[3] + observation[4])/2))*max((300 - observation[2])/300, 0)/750
    return observation, reward, done, info

  def reset(self):
    self.env.reset()
    observation = np.array(list(self.env.game_state.getGameState().values()))
    return observation  # reward, done, info can't be included

  def render(self, mode='human'):
    self.env.render(mode='human')
    
  def close (self):
    self.env.close()

Checando se o nosso ambiente satisfaz as propriedades do Gym.

In [6]:
from stable_baselines.common.env_checker import check_env

env = FlappyBirdEnv()
check_env(env)

D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Environment '<class 'gym_ple.ple_env.PLEEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Criando um vetor com 16 ambientes para multiprocessamento.

In [7]:
import gym

from stable_baselines.common import set_global_seeds, make_vec_env

env = make_vec_env(FlappyBirdEnv, n_envs = 16)

### Agente

Criando o modelo de PPO2 com a biblioteca Stable Baselines.

In [14]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import PPO2

model = PPO2(MlpPolicy, env, n_steps = 512, nminibatches = 64, lam = 0.98, gamma = 0.99, noptepochs= 10, ent_coef= 0.001, verbose=1)

Você tem a opção de usar um modelo pré-treinado:

In [8]:
from stable_baselines import PPO2

model = PPO2.load("trained_models/PPO2_1280")
model.set_env(env)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading a model without an environment, this model cannot be trained until it has a valid environment.










#### Treinamento do Modelo:

In [6]:
# model.learn(total_timesteps=100000)
model.learn(total_timesteps=8192)

---------------------------------------
| approxkl           | 0.012391278    |
| clipfrac           | 0.081848145    |
| ep_len_mean        | 200            |
| ep_reward_mean     | 23.1           |
| explained_variance | 0.289          |
| fps                | 313            |
| n_updates          | 1              |
| policy_entropy     | 0.29807568     |
| policy_loss        | -0.00067299267 |
| serial_timesteps   | 512            |
| time_elapsed       | 0              |
| total_timesteps    | 8192           |
| value_loss         | 1.3809203      |
---------------------------------------


#### Testando o Modelo: 

In [9]:
TestEnv = FlappyBirdEnv()

D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [10]:
obs = TestEnv.reset()
dones = 0
while not dones:
    TestEnv.render()
    action, _states = model.predict(obs)
    obs, rewards, dones, info = TestEnv.step(action)
    

TestEnv.close()

#### Salvando o Modelo:

In [60]:
model.save("trained_models/PPO2_1280")

#### Gravando um Episódio: (Not Working)

Criando o ambiente gravado.

In [11]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

video_folder = 'videos/'
video_length = 100

VideoEnv = DummyVecEnv([lambda: FlappyBirdEnv()])

obs = VideoEnv.reset()

VideoEnv = VecVideoRecorder(VideoEnv, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="FlappyBird")

Rodando o episódio.

In [13]:
obs = VideoEnv.reset()

for _ in range(video_length + 1):
    VideoEnv.render()
    action, _ = model.predict(obs)
    obs, _, _, _ = VideoEnv.step(action)

VideoEnv.close()


D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Env returned None on render(). Disabling further rendering for video recorder by marking as disabled: path=D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.mp4 metadata_path=D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.meta.json
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Saving video to  D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.mp4


## Flappy Bird com Visão Computacional

A implementar

In [11]:
ENV_NAME = "FlappyBird-v0"

import gym

from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

env_vision = gym.make(ENV_NAME)
# Optional: PPO2 requires a vectorized environment to run
# the env is now wrapped automatically when passing it to the constructor
env_vision = DummyVecEnv([lambda: env_vision])

In [ ]:
modelVision = PPO2(CnnLstmPolicy, env_vision, n_steps = 512, nminibatches = 1, lam = 0.98, gamma = 0.999, noptepochs= 15, ent_coef= 0.01, verbose=1)

In [ ]:
modelVision.learn(total_timesteps=10000)

In [ ]:
obs = env_vision.reset()
dones = 0
while not dones:
    action, _states = modelVision.predict(obs)
    obs, rewards, dones, info = env_vision.step(action)
    env.render()

env_vision.close()